In [7]:
# –> Import required packages
# –> If the packages have not been installed yet, you might want to go to 
#    https://anaconda.org to search for the package you need, and install
#    before importing
import os
import numpy as np
import pandas as pd
from pyteomics import mzml
import time
import more_itertools as mit

# Step 0: Put the files into the same path
# Change the path and filename
######################################################
### NEED TO CHANGE DIRECTORY AND FILE NAMES HERE
######################################################
# Directory only! Don't include filename
file_path = os.path.abspath("D:\\0.2TMT fragmentation\\20220724\\1_mzML")
# file name only! Don't keep directory
mzml_filename = "20220722_YG_HelaTMT4_5ug_HCD35_45_50_3.mzML"
######################################################
### DONT CHANGE BELOW ################################

start_time1 = time.time()
with mzml.read(os.path.join(file_path,mzml_filename)) as reader:
    spectrum_list = []
    scan_id = []
    index = []
    intensity = []
    mz = []
    ms_cat = []
    for spectrum in reader:
        spectrum_list.append(spectrum)
        scan_id.append(int((spectrum['id'].split()[-1]).split('=')[-1]))
        index.append(spectrum['index'])
        intensity.append(spectrum['intensity array'])
        mz.append(spectrum['m/z array'])
        ms_cat.append(spectrum['ms level'])
        
## Filter where mz have values between 126.127-131.1385, values can be adjusted for differnt TMTplex reagents
imz_list = []
mz_new = []
intensity_new = []
for imz,mz_array in enumerate(mz):
    # Filter out unnecessary info to shrink the size of mz / intensity matrix
    index_inrange = np.where((mz_array>=126.127)&(mz_array<=131.1385))[0]
    if index_inrange.tolist() != []:
        imz_list.append(imz)
        mz_new.append(mz_array[index_inrange])
        intensity_new.append(intensity[imz][index_inrange])
        
max_columns = np.max([len(x) for x in mz_new])     
mz_m = np.zeros((len(imz_list),max_columns),dtype=np.dtype('f4'))
intensity_m = np.zeros((len(imz_list),max_columns),dtype=np.dtype('f4'))
for i in range(len(imz_list)):
    mz_m[i,:len(mz_new[i])] = mz_new[i]
    intensity_m[i,:len(intensity_new[i])] = intensity_new[i]
end_time1 = time.time()
print("finished!")

start_time2 = time.time()
## Search for mz in the given ranges
## 12x : 126-131
lower_bounds = [126.127, 127.124, 128.134, 129.131, 130.1405, 131.1375]
interval = 0.001
mtx_12x = np.zeros((len(scan_id),len(lower_bounds)),dtype=np.dtype('f4'))
for i12x in range(len(lower_bounds)):
    list_12x = np.zeros(len(imz_list),dtype=np.dtype('f4'))
    indx_12x = np.where((mz_m>lower_bounds[i12x])&(mz_m<lower_bounds[i12x]+interval))
    # indx_12x[0]: rows of values in range;
    # indx_12x[1]: corresponding columns
    ind_12x_rows = indx_12x[0]
    if ind_12x_rows.tolist() != []:
        for irow in set(ind_12x_rows):
            columns = indx_12x[1]
            icolumn = columns[np.where(ind_12x_rows==irow)[0]]
            iintensity = intensity_m[irow,icolumn[0]:icolumn[-1]+1]
            list_12x[irow] = np.max(iintensity)
    mtx_12x[imz_list,i12x] = list_12x
    
## Write into a dataframe and export as csv file
ms1 = {"ID_MS1": index,
      "SCAN": scan_id,
      "126":mtx_12x[:,0],
      "127":mtx_12x[:,1],
      "128":mtx_12x[:,2],
      "129":mtx_12x[:,3],
      "130":mtx_12x[:,4],
      "131":mtx_12x[:,5],
      "Category":ms_cat
      }
df_ms1 = pd.DataFrame(ms1)
df_ms1.set_index("ID_MS1")
df_ms1.to_csv(os.path.join(file_path,'TMT_'+mzml_filename.split('.')[0]+'.csv'),index=False)

print("-------------%s seconds-------------" % (time.time()-start_time2+(end_time1-start_time1)))
print("finished!")

finished!
-------------12.155727624893188 seconds-------------
finished!
